### Data used: https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [181]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

In [182]:
with open("../data/shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [183]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [184]:
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


### Encoder and Decoder section

#### Further development idea:
Use Sentencepiece (https://github.com/google/sentencepiece) or Tiktoken (https://github.com/openai/tiktoken) intead of the following mapping

In [185]:
stoi = {
    ch:i for i, ch in enumerate(chars)
    }

itos = {
    i:ch for i, ch in enumerate(chars)
    }

In [186]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [187]:
print(encode("Encode this"))
print(decode(encode("Encode this")))

[17, 52, 41, 53, 42, 43, 1, 58, 46, 47, 57]
Encode this


#### Encoding the entire dataset

In [188]:
data = torch.tensor(encode(text), dtype=torch.long)

In [189]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [190]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [191]:
block_size = 8
batch_size = 4
train_data[:block_size + 1]

torch.manual_seed = 9876

In [192]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i+block_size + 1] for i in ix])

    return x, y

In [193]:
xb, yb = get_batch("train")

In [194]:
class BiagramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None): #B, T, C = Batch, Time, Channel
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) #Pytorch cross_entropy expects B, T and C in this way when dealing with multi dimensional structures
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [195]:
model = BiagramLanguageModel(vocab_size)
logits, loss = model(xb, yb)

print(logits.shape)

torch.Size([32, 65])


In [196]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


?bm?!Yt!.qO.b'w-
avzG$EW HINEoHM-H'GeiZctG,Cf mH'
a'kXgwdcrtOroEof$?c
?kCCdKOgRYdpoHDAe3uhnJk;CMF,vj


In [197]:
print(loss)

tensor(4.7411, grad_fn=<NllLossBackward0>)


### Negative log likelyhood

In [198]:
math.log(1/vocab_size) * -1

4.174387269895637

Expecting ~4.17 loss

In [199]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    #Evaluating the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.493596315383911


#### Let's check the improvement

In [200]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))


HRTh psunowimavin I criteso erondarere-HNGRIORIORY myo bllto pinthe, sor:
Whe haie sew, te! ccanerd; NTreatan:

Thepulalurorve,

ARache menoro intioust t chelicu cod m cer'Thesbes pt tha t--thee him! h the mit
TES:

Whintharerd athidit, t aitod nck tlemy,
D pusthey be.
I y end ar?

DUSires nt!3anowh bead ouxe t nourino wno bricouel nory courcotee pie t thente d mar he me, rorad:
Tinthinied
I we S:
CHENo s coonssite?
WERen thy tot tan,
Hengowh ceinost ilotheaved s!'d; juser mpal athaien wee, htar


#### No numbers is a good start

In [201]:
B, T, C = 4, 8, 2

x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [202]:
xbow = torch.zeros(B, T, C)

for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]
        xbow[b, t] = torch.mean(xprev, 0)

In [203]:
x[0]

tensor([[ 0.7162,  0.5406],
        [-0.9009, -0.1400],
        [ 1.1242,  0.8962],
        [-1.4982, -1.0826],
        [-1.6056, -0.1401],
        [-0.7585,  0.3850],
        [ 0.7621,  0.4532],
        [ 1.1639,  1.5736]])

Nested Loops are not efficient, but we can utilise matrix multiplication to achieve more efficient way for self attention (i.e. information about the previous tokens).

#### Toy example

In [204]:
a= torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b

print("________________________________")
print("a==")
print(a)
print("________________________________")
print("b==")
print(b)
print("________________________________")
print("c==")
print(c)
print("________________________________")

________________________________
a==
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
________________________________
b==
tensor([[8., 1.],
        [1., 9.],
        [9., 1.]])
________________________________
c==
tensor([[8.0000, 1.0000],
        [4.5000, 5.0000],
        [6.0000, 3.6667]])
________________________________


#### Toy example ends

### First version

In [205]:
#wei = torch.tril(torch.ones(T, T))
#wei = wei / wei.sum(1, keepdim=True)

#xbow2 = wei @ x

### Second version

In [213]:
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)
q = query(x)
wei = q @ k.transpose(-2, -1)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim = -1)

v = value(x)
out = wei @ v

In [214]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.9163, 0.0837, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4397, 0.4049, 0.1554, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2204, 0.5045, 0.1277, 0.1475, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2100, 0.3030, 0.0306, 0.0105, 0.4459, 0.0000, 0.0000, 0.0000],
        [0.0606, 0.0454, 0.3672, 0.3597, 0.0326, 0.1345, 0.0000, 0.0000],
        [0.0813, 0.0938, 0.2175, 0.0281, 0.0523, 0.1626, 0.3644, 0.0000],
        [0.0809, 0.1696, 0.1249, 0.0608, 0.0752, 0.3066, 0.0975, 0.0844]],
       grad_fn=<SelectBackward0>)